# Internal Feedback Loop for Prompt & Model Optimization

## 📋 Purpose

**Goal**: Build an internal evaluation pipeline that scores model outputs and provides insights to improve prompts, guardrails, and model selection.

**This is NOT user-facing** — it's a development tool for iterating on our simplification system.

---

## 🔄 The Feedback Loop

```
┌─────────────────────────────────────────────────────────────┐
│                    INTERNAL FEEDBACK LOOP                    │
├─────────────────────────────────────────────────────────────┤
│                                                              │
│   1. RUN BATCH          2. SCORE OUTPUTS      3. ANALYZE    │
│   ─────────────         ──────────────        ──────────    │
│   Source texts    →     - ARI score      →    - Which       │
│   + Current prompts     - Sentence len        prompts work? │
│   + Selected model      - Meaning sim         - Where do    │
│                                               models fail?  │
│                              ↓                              │
│                                                              │
│   4. ADJUST             ←─────────────────────────────────  │
│   ────────                                                   │
│   - Tweak prompts                                           │
│   - Update guardrails                                       │
│   - Try different model                                     │
│   - Repeat                                                  │
│                                                              │
└─────────────────────────────────────────────────────────────┘
```

---

## 🎯 Sprint Plan (10 days, 1 person)

### Sprint 1: Scoring Pipeline (Days 1-5)
| Day | Task | Output |
|-----|------|--------|
| 1-2 | Batch scoring script | Score all test texts |
| 2-3 | Results logging | CSV/JSONL with scores per text |
| 4-5 | Analysis notebook | Aggregate stats, identify patterns |

### Sprint 2: Iteration Tools (Days 6-10)
| Day | Task | Output |
|-----|------|--------|
| 6-7 | Prompt comparison | Compare prompt variants |
| 8-9 | Model comparison | Compare model outputs |
| 9-10 | Guardrail tuning | Adjust thresholds based on data |

---

## ⚙️ Acceptance Criteria

1. ✅ Can batch-score all test texts with current prompt/model
2. ✅ Results logged to file for analysis
3. ✅ Can compare metrics across prompt variants
4. ✅ Insights feed back into prompt/guardrail improvements

---

# 📚 Implementation Guide

## 🔧 Prerequisites

- Python 3.10+
- `pip install sentence-transformers scikit-learn pandas`
- Test texts in `data/`
- Model API access (Groq/OpenAI)

In [ ]:
# Setup: Install required packages if needed
# Uncomment and run if dependencies are missing

# !pip install sentence-transformers spacy scikit-learn pyyaml
# !python -m spacy download de_core_news_sm
# !python -m spacy download en_core_web_sm

---

# 🚀 Sprint 1: Scoring Pipeline (Days 1-5)

## Task 1.1: Batch Scoring Setup (Days 1-2)

**Goal**: Score model outputs in batch, log results for analysis.

### What We're Building

```
test_texts.csv → Run through model → Score each output → results.csv
```

### Metrics to Track
| Metric | What it tells us |
|--------|------------------|
| `ari_score` | Is output readable? (target: ≤8) |
| `avg_sentence_len` | Are sentences short enough? (target: ≤15) |
| `meaning_cosine` | Did we preserve meaning? (target: ≥0.70) |

In [1]:
# Task 1.1: Batch scoring function
# This scores a single text and returns metrics as a dict

import re
import numpy as np

def compute_ari(text: str) -> float:
    """Compute Automated Readability Index (works for EN + DE)."""
    sentences = [s.strip() for s in re.split(r'[.!?]+', text) if s.strip()]
    words = re.findall(r'\b\w+\b', text)
    if not sentences or not words:
        return 0.0
    chars = sum(len(w) for w in words)
    return 4.71 * (chars / len(words)) + 0.5 * (len(words) / len(sentences)) - 21.43

def compute_avg_sentence_len(text: str) -> float:
    """Compute average words per sentence."""
    sentences = [s.strip() for s in re.split(r'[.!?]+', text) if s.strip()]
    words = re.findall(r'\b\w+\b', text)
    if not sentences:
        return 0.0
    return len(words) / len(sentences)

def compute_meaning_similarity(source: str, output: str) -> float:
    """Compute cosine similarity using sentence-transformers."""
    try:
        from sentence_transformers import SentenceTransformer
        model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
        embeddings = model.encode([source, output])
        sim = np.dot(embeddings[0], embeddings[1]) / (
            np.linalg.norm(embeddings[0]) * np.linalg.norm(embeddings[1])
        )
        return float(sim)
    except ImportError:
        print("Warning: sentence-transformers not installed")
        return 0.0

def score_output(source: str, output: str) -> dict:
    """Score a single simplification. Returns dict of metrics."""
    return {
        "ari_score": round(compute_ari(output), 1),
        "avg_sentence_len": round(compute_avg_sentence_len(output), 1),
        "meaning_cosine": round(compute_meaning_similarity(source, output), 3),
    }

# Test
test_source = "Die meteorologischen Bedingungen sind ungünstig."
test_output = "Das Wetter ist schlecht."
print("Test scores:", score_output(test_source, test_output))

/Users/simonvoegely/Desktop/easysprache/klartext/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Test scores: {'ari_score': 4.1, 'avg_sentence_len': 4.0, 'meaning_cosine': 0.792}


---

## Task 1.2: Results Logging (Days 2-3)

**Goal**: Log all scores to CSV for later analysis.

### Output Format
```
source_id, prompt_version, model, ari_score, avg_sentence_len, meaning_cosine, passed
```

In [2]:
# Task 1.2: Batch evaluation and logging
import pandas as pd
from datetime import datetime

# Guardrail thresholds
THRESHOLDS = {
    "ari_score": {"max": 8, "direction": "lower"},
    "avg_sentence_len": {"max": 15, "direction": "lower"},
    "meaning_cosine": {"min": 0.70, "direction": "higher"},
}

def check_passed(scores: dict) -> bool:
    """Check if all metrics pass thresholds."""
    return (
        scores["ari_score"] <= THRESHOLDS["ari_score"]["max"] and
        scores["avg_sentence_len"] <= THRESHOLDS["avg_sentence_len"]["max"] and
        scores["meaning_cosine"] >= THRESHOLDS["meaning_cosine"]["min"]
    )

def run_batch_evaluation(test_data: list, prompt_version: str, model_name: str) -> pd.DataFrame:
    """
    Run batch evaluation on test data.
    
    Args:
        test_data: List of dicts with 'id', 'source', 'output' keys
        prompt_version: e.g., "v1.2"
        model_name: e.g., "llama-3.1-8b-instant"
    
    Returns:
        DataFrame with all results
    """
    results = []
    
    for item in test_data:
        scores = score_output(item["source"], item["output"])
        
        results.append({
            "source_id": item["id"],
            "prompt_version": prompt_version,
            "model": model_name,
            "ari_score": scores["ari_score"],
            "avg_sentence_len": scores["avg_sentence_len"],
            "meaning_cosine": scores["meaning_cosine"],
            "passed": check_passed(scores),
            "timestamp": datetime.now().isoformat(),
        })
    
    return pd.DataFrame(results)

# Example usage
test_data = [
    {"id": "test_001", "source": "Die Situation ist kompliziert.", "output": "Das ist schwer."},
    {"id": "test_002", "source": "Bitte beachten Sie die Hinweise.", "output": "Lesen Sie die Tipps."},
]

df = run_batch_evaluation(test_data, prompt_version="v1.0", model_name="llama-3.1-8b")
print(df.to_string(index=False))

source_id prompt_version        model  ari_score  avg_sentence_len  meaning_cosine  passed                  timestamp
 test_001           v1.0 llama-3.1-8b       -1.1               3.0           0.750    True 2026-01-12T08:35:38.740568
 test_002           v1.0 llama-3.1-8b       -0.6               4.0           0.729    True 2026-01-12T08:35:41.948790


---

## Task 1.3: Analysis Functions (Days 4-5)

**Goal**: Aggregate results and identify patterns that inform prompt/guardrail changes.

In [3]:
# Task 1.3: Analysis functions

def analyze_results(df: pd.DataFrame) -> dict:
    """
    Analyze batch results and return insights.
    
    Returns dict with:
    - pass_rate: % of texts that pass all thresholds
    - metric_means: average for each metric
    - failures: which metrics fail most often
    """
    analysis = {
        "total_texts": len(df),
        "pass_rate": df["passed"].mean() * 100,
        "metric_means": {
            "ari_score": df["ari_score"].mean(),
            "avg_sentence_len": df["avg_sentence_len"].mean(),
            "meaning_cosine": df["meaning_cosine"].mean(),
        },
        "failure_rates": {
            "ari_too_high": (df["ari_score"] > 8).mean() * 100,
            "sentences_too_long": (df["avg_sentence_len"] > 15).mean() * 100,
            "meaning_lost": (df["meaning_cosine"] < 0.70).mean() * 100,
        }
    }
    return analysis

def generate_feedback_report(analysis: dict) -> str:
    """Generate actionable feedback for prompt/guardrail tuning."""
    lines = ["# Evaluation Report\n"]
    
    lines.append(f"**Pass Rate**: {analysis['pass_rate']:.0f}%\n")
    
    lines.append("## Failure Analysis\n")
    for issue, rate in analysis["failure_rates"].items():
        if rate > 20:  # Flag issues affecting >20% of texts
            lines.append(f"⚠️ **{issue}**: {rate:.0f}% of outputs")
    
    lines.append("\n## Recommendations\n")
    
    if analysis["failure_rates"]["ari_too_high"] > 20:
        lines.append("- **Prompt**: Add instruction to use simpler words")
    
    if analysis["failure_rates"]["sentences_too_long"] > 20:
        lines.append("- **Prompt**: Add instruction to keep sentences under 15 words")
    
    if analysis["failure_rates"]["meaning_lost"] > 20:
        lines.append("- **Prompt**: Add instruction to preserve all key information")
    
    return "\n".join(lines)

# Test with sample data
analysis = analyze_results(df)
print(generate_feedback_report(analysis))

# Evaluation Report

**Pass Rate**: 100%

## Failure Analysis


## Recommendations



---

# 🚀 Sprint 2: Comparison & Iteration Tools (Days 6-10)

## Task 2.1: Prompt Comparison (Days 6-7)

**Goal**: Compare different prompt versions to see which performs better.

In [ ]:
# Task 2.1: Compare prompt versions

def compare_prompts(results_df: pd.DataFrame) -> pd.DataFrame:
    """
    Compare metrics across different prompt versions.
    
    Returns DataFrame grouped by prompt_version with aggregated metrics.
    """
    comparison = results_df.groupby("prompt_version").agg({
        "ari_score": "mean",
        "avg_sentence_len": "mean", 
        "meaning_cosine": "mean",
        "passed": "mean"
    }).round(2)
    
    comparison.columns = ["Avg ARI", "Avg Sent Len", "Avg Meaning", "Pass Rate"]
    comparison["Pass Rate"] = (comparison["Pass Rate"] * 100).astype(int).astype(str) + "%"
    
    return comparison

# Example: Results from two prompt versions
sample_results = pd.DataFrame([
    {"prompt_version": "v1.0", "model": "llama", "ari_score": 9.2, "avg_sentence_len": 16, "meaning_cosine": 0.75, "passed": False},
    {"prompt_version": "v1.0", "model": "llama", "ari_score": 8.5, "avg_sentence_len": 14, "meaning_cosine": 0.72, "passed": False},
    {"prompt_version": "v1.1", "model": "llama", "ari_score": 6.8, "avg_sentence_len": 12, "meaning_cosine": 0.78, "passed": True},
    {"prompt_version": "v1.1", "model": "llama", "ari_score": 7.2, "avg_sentence_len": 11, "meaning_cosine": 0.81, "passed": True},
])

print("PROMPT COMPARISON")
print("="*50)
print(compare_prompts(sample_results))
print("\n→ v1.1 performs better: lower ARI, higher pass rate")

---

## Task 2.2: Model Comparison (Days 8-9)

**Goal**: Compare how different models perform on the same test set.

In [ ]:
# Task 2.2: Compare models

def compare_models(results_df: pd.DataFrame) -> pd.DataFrame:
    """Compare metrics across different models."""
    comparison = results_df.groupby("model").agg({
        "ari_score": "mean",
        "avg_sentence_len": "mean",
        "meaning_cosine": "mean",
        "passed": "mean"
    }).round(2)
    
    comparison.columns = ["Avg ARI", "Avg Sent Len", "Avg Meaning", "Pass Rate"]
    comparison["Pass Rate"] = (comparison["Pass Rate"] * 100).astype(int).astype(str) + "%"
    
    return comparison.sort_values("Pass Rate", ascending=False)

# Example: Results from different models
model_results = pd.DataFrame([
    {"model": "llama-3.1-8b", "ari_score": 7.5, "avg_sentence_len": 12, "meaning_cosine": 0.78, "passed": True},
    {"model": "llama-3.1-8b", "ari_score": 8.1, "avg_sentence_len": 14, "meaning_cosine": 0.75, "passed": True},
    {"model": "gpt-4o-mini", "ari_score": 6.2, "avg_sentence_len": 10, "meaning_cosine": 0.82, "passed": True},
    {"model": "gpt-4o-mini", "ari_score": 5.8, "avg_sentence_len": 9, "meaning_cosine": 0.85, "passed": True},
    {"model": "mixtral-8x7b", "ari_score": 9.5, "avg_sentence_len": 18, "meaning_cosine": 0.71, "passed": False},
    {"model": "mixtral-8x7b", "ari_score": 10.2, "avg_sentence_len": 20, "meaning_cosine": 0.68, "passed": False},
])

print("MODEL COMPARISON")
print("="*50)
print(compare_models(model_results))
print("\n→ Use this to select best model for production")

---

## Task 2.3: Guardrail Tuning (Days 9-10)

**Goal**: Adjust thresholds based on what the data shows is realistic.

In [ ]:
# Task 2.3: Guardrail tuning based on results

def suggest_guardrail_updates(results_df: pd.DataFrame) -> dict:
    """
    Suggest guardrail threshold adjustments based on actual results.
    
    If pass rate is too low, thresholds may be too strict.
    If pass rate is 100%, thresholds may be too loose.
    """
    suggestions = {}
    
    pass_rate = results_df["passed"].mean()
    
    if pass_rate < 0.5:
        # Too few passing - analyze which metric fails most
        ari_fail_rate = (results_df["ari_score"] > 8).mean()
        sent_fail_rate = (results_df["avg_sentence_len"] > 15).mean()
        meaning_fail_rate = (results_df["meaning_cosine"] < 0.70).mean()
        
        if ari_fail_rate > 0.3:
            new_threshold = results_df["ari_score"].quantile(0.8)
            suggestions["ari_score"] = f"Consider relaxing to {new_threshold:.0f} (currently 8)"
        
        if sent_fail_rate > 0.3:
            new_threshold = results_df["avg_sentence_len"].quantile(0.8)
            suggestions["avg_sentence_len"] = f"Consider relaxing to {new_threshold:.0f} (currently 15)"
            
        if meaning_fail_rate > 0.3:
            new_threshold = results_df["meaning_cosine"].quantile(0.2)
            suggestions["meaning_cosine"] = f"Consider lowering to {new_threshold:.2f} (currently 0.70)"
    
    elif pass_rate == 1.0:
        suggestions["general"] = "100% pass rate - consider tightening thresholds"
    
    return suggestions

# Example
print("GUARDRAIL TUNING SUGGESTIONS")
print("="*50)
suggestions = suggest_guardrail_updates(model_results)
if suggestions:
    for metric, suggestion in suggestions.items():
        print(f"• {metric}: {suggestion}")
else:
    print("✅ Current thresholds look good (pass rate is reasonable)")

---

# 📋 Complete Workflow Example

## Running a Full Iteration Cycle

In [ ]:
# Complete workflow: One iteration cycle

def run_iteration(test_texts: list, prompt_version: str, model: str):
    """
    Run one complete iteration of the feedback loop.
    
    1. Score all outputs
    2. Analyze results
    3. Generate recommendations
    """
    print(f"\n{'='*60}")
    print(f"ITERATION: {prompt_version} + {model}")
    print(f"{'='*60}")
    
    # Step 1: Score
    results = run_batch_evaluation(test_texts, prompt_version, model)
    
    # Step 2: Analyze
    analysis = analyze_results(results)
    
    print(f"\n📊 RESULTS:")
    print(f"   Pass Rate: {analysis['pass_rate']:.0f}%")
    print(f"   Avg ARI: {analysis['metric_means']['ari_score']:.1f}")
    print(f"   Avg Sent Len: {analysis['metric_means']['avg_sentence_len']:.1f}")
    print(f"   Avg Meaning: {analysis['metric_means']['meaning_cosine']:.2f}")
    
    # Step 3: Recommendations
    print(f"\n💡 RECOMMENDATIONS:")
    if analysis['failure_rates']['ari_too_high'] > 20:
        print("   → Prompt: Emphasize using simpler, shorter words")
    if analysis['failure_rates']['sentences_too_long'] > 20:
        print("   → Prompt: Add rule 'max 15 words per sentence'")
    if analysis['failure_rates']['meaning_lost'] > 20:
        print("   → Prompt: Add 'preserve all key information'")
    if analysis['pass_rate'] >= 80:
        print("   → ✅ This configuration is working well!")
    
    return results

# Example iteration
test_texts = [
    {"id": "001", "source": "Die Anmeldung muss rechtzeitig erfolgen.", 
     "output": "Melden Sie sich früh an."},
    {"id": "002", "source": "Beachten Sie die Öffnungszeiten.", 
     "output": "Wir sind zu bestimmten Zeiten offen."},
]

run_iteration(test_texts, "v1.0", "llama-3.1-8b")

---

## Saving Results for Tracking Progress

In [ ]:
# Save results to CSV for tracking across iterations
import os

def save_iteration_results(results_df: pd.DataFrame, output_dir: str = "../outputs"):
    """Append results to tracking CSV."""
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, "iteration_results.csv")
    
    if os.path.exists(output_path):
        # Append to existing
        existing = pd.read_csv(output_path)
        combined = pd.concat([existing, results_df], ignore_index=True)
        combined.to_csv(output_path, index=False)
    else:
        results_df.to_csv(output_path, index=False)
    
    print(f"✅ Results saved to {output_path}")

# Example
# save_iteration_results(results_df)
print("Use save_iteration_results(df) to log each iteration")

---

# ✅ Done Checklist

In [ ]:
checklist = """
SPRINT 1: Scoring Pipeline
[ ] score_output() function works
[ ] run_batch_evaluation() scores all test texts
[ ] analyze_results() identifies patterns
[ ] generate_feedback_report() produces actionable recommendations

SPRINT 2: Comparison & Iteration
[ ] compare_prompts() shows which prompt version is better
[ ] compare_models() shows which model performs best
[ ] suggest_guardrail_updates() recommends threshold changes
[ ] save_iteration_results() logs progress over time

THE LOOP IS WORKING WHEN:
→ You can run a batch, see what's failing, adjust prompts/guardrails, and re-run
→ Each iteration shows measurable improvement (or confirms current setup is optimal)
"""
print(checklist)

---

## 🔄 How to Use This Feedback Loop

1. **Prepare test set**: Collect source texts to simplify
2. **Run model**: Generate outputs with current prompt/model
3. **Score batch**: Use `run_batch_evaluation()`
4. **Analyze**: Use `analyze_results()` and `generate_feedback_report()`
5. **Adjust**: Modify prompts or guardrails based on recommendations
6. **Repeat**: Run again and compare with `compare_prompts()`

In [ ]:
# Example: Full iteration cycle with logging

print("EXAMPLE: Running feedback loop iteration")
print("="*50)
print("""
# 1. Prepare test data (source + model outputs)
test_data = load_test_outputs("outputs/model_run_v1.csv")

# 2. Score batch
results = run_batch_evaluation(test_data, "prompt_v1.2", "llama-3.1-8b")

# 3. Analyze
analysis = analyze_results(results)
print(generate_feedback_report(analysis))

# 4. If pass rate < target, adjust prompts and re-run
# 5. Compare: compare_prompts(all_results_df)

# 6. Save for tracking
save_iteration_results(results)
""")

---

## Key Metrics Explained

| Metric | What it measures | Target | If failing... |
|--------|------------------|--------|---------------|
| `ari_score` | Text complexity | ≤8 | Use simpler words |
| `avg_sentence_len` | Sentence length | ≤15 | Split long sentences |
| `meaning_cosine` | Semantic similarity | ≥0.70 | Preserve key info |

In [ ]:
# Thresholds - adjust based on your analysis
THRESHOLDS = {
    "ari_score": 8,          # Max ARI (grade level)
    "avg_sentence_len": 15,   # Max words per sentence
    "meaning_cosine": 0.70,   # Min semantic similarity
}

print("Current thresholds:")
for metric, value in THRESHOLDS.items():
    direction = "≥" if metric == "meaning_cosine" else "≤"
    print(f"  {metric}: {direction} {value}")

---

## Summary: The Internal Feedback Loop

```
┌────────────┐    ┌────────────┐    ┌────────────┐    ┌────────────┐
│   RUN      │ →  │   SCORE    │ →  │  ANALYZE   │ →  │   ADJUST   │
│  Model +   │    │  Outputs   │    │  Results   │    │  Prompts/  │
│  Prompt    │    │            │    │            │    │ Guardrails │
└────────────┘    └────────────┘    └────────────┘    └─────┬──────┘
                                                           │
      ┌────────────────────────────────────────────────────┘
      │
      ▼
   REPEAT
```

In [ ]:
print("🎯 This feedback loop helps you iteratively improve:")
print("   • Prompts: What instructions make the model produce better output?")
print("   • Models: Which model works best for Easy Language?")
print("   • Guardrails: What thresholds are realistic for your use case?")

---

*This is an internal development tool. The insights from this loop inform changes to prompts and guardrails—changes that you implement manually based on the data.*